In [ ]:
import pathlib
import tensorflow as tf
from tensorflow import keras
from keras import layers
from matplotlib import pyplot as plt 
# import cv2
# import numpy as np

In [ ]:
img_height = 224
img_width = 224
input_shape = (img_width, img_height, 3)
batch_size = 32
train_dir = pathlib.Path('../input/cacao-augmented-224/training_img/training_img/')
test_dir  = pathlib.Path('../input/cacao-augmented-224/testing_img/testing_img/')
checkpoint_dir = pathlib.Path('./cacao_CAE/checkpoint')
model_dir = pathlib.Path('./cacao_CAE/model')


In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
  train_dir,
  labels=None,
  shuffle=True,
  image_size=(img_height, img_width),
  batch_size=batch_size
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)


In [ ]:
input_layer = layers.Input(shape=input_shape)
input_layer2 = layers.Rescaling(scale=1./255, offset=0)(input_layer)
# encoder
x = layers.Conv2D(112, kernel_size=(3,3), activation="relu", padding="same")(input_layer2)

x = layers.Conv2D(96, kernel_size=(5,5), activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

x = layers.Conv2D(28, kernel_size=(5,5), activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

x = layers.Conv2D(14, kernel_size=(3,3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

x = layers.Flatten()(x)
x = layers.Dense(4096, activation="relu")(x)
x = layers.Dense(1024, activation="relu")(x)
# #latent
latent = layers.Dense(56, activation="sigmoid")(x)
# #decoder
x = layers.Dense(1024, activation="relu")(latent)
x = layers.Dense(4096, activation="relu")(x)
x = layers.Dense(2352, activation="relu")(x)
x = layers.Reshape((14,14,12))(x)
x = layers.Conv2DTranspose(14, kernel_size=(3,3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(28, kernel_size=(5,5), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(96, kernel_size=(5,5), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(112, kernel_size=(3,3), strides=2, activation="relu", padding="same")(x)

output_layer = layers.Rescaling(scale=255, offset=0)(x)
output_layer = layers.Conv2D(3, kernel_size=(3,3), padding="same")(output_layer)

model = keras.Model(input_layer, output_layer)
# model.summary()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model = keras.models.load_model(model_dir)

In [ ]:
from os import walk
for (dirpath, dirnames, filenames) in walk("../input"):
    print("Directory path: ", dirpath)
    print("Folder name: ", dirnames)
#     print("File name: ", filenames)

In [ ]:
epochs = 1
i=0
j=0
for ep in range(epochs):
  j = j+1
  print("Epoch",j, "/", epochs)
  for batch in train_ds.take(500):
      i = i+1
      score = model.train_on_batch(x=batch, y=batch, reset_metrics=False, return_dict=False)
      print("batch",i, "--Loss:",score[0])

model.save(model_dir)

In [ ]:
model.save(model_dir)

In [ ]:
for img in train_ds.take(1):
    image = img
    break

result = model.predict(image, batch_size=None)

# print(np.shape(result))
image = np.asarray(image[0], dtype="uint8")
plt.imshow(image)

result = np.asarray(result[0], dtype="uint8")
plt.imshow(result)
